In [50]:
import tweepy
import csv
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
plt.style.use('seaborn')
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10,9
sb.set_style('whitegrid')

# Authenticate to Twitter
customer_key = "cht3zkzkhKtCXadYO8a7fCa4E"
customer_key_secret = "oN4KLrQOqYjKjJpS7b8C7kvKhNpCSrMOYT1vxjjciWmGGxGKmJ"
access_token = "1300264439929663488-Vy4BHsechVLPxB48tksIfTMdt7S8Mr"
access_token_secret = "BafJQS8QHxSK8v7NF5hSMS9PmrSmsCuM1ZSVSmoZ24uBu"
    
auth = tweepy.OAuthHandler(customer_key, customer_key_secret)
auth.set_access_token(access_token,access_token_secret)

# Create API object
api = tweepy.API(auth)

try:
        api.verify_credentials()
        print("Authentication OK")
except:
        print("Error during authentication")
    
tweet = pd.DataFrame([tweet._json['text'] for tweet in tweepy.Cursor(api.user_timeline, screen_name = 'BillGates').items()]
                         ,columns = ['tweet'])

Authentication OK


In [52]:
pd.options.display.max_colwidth = 200
tweet.shape
tweet

,tweet
0,"Here in Washington, we always vote by mail. It's a safe and secure way to shape the future of our country. But rega… https://t.co/wGSCJYxxY1"
1,@MohamedBinZayed The UAE is an essential partner in stopping polio in Pakistan. Thank you @MohamedBinZayed for your… https://t.co/5xQEZ8CpUL
2,"For decades, @Rotary has worked tirelessly to #EndPolio. Today, the health infrastructure they’ve built around the… https://t.co/vYYV7djNsE"
3,"RT @StephenCurry30: Even with his busy schedule, Dr. Fauci took the time to sit down with me (AGAIN) and talk about what we’ve gotten right…"
4,I’ve been working for some time on a book about what we need to do over the next decade to avoid a climate disaster… https://t.co/UvV6cRjxOW
...,...
3195,Students from KIPP Academy in the Bronx produced a video with questions. I answered some of them on Gates Notes - http://bit.ly/aBhI21
3196,Melinda's TEDx talk has posted on the TED site - http://bit.ly/bWlsyD
3197,"My notes on “Energy Transitions” - http://bit.ly/bF6pTr - a book by Vaclav Smil, one of my favorite writers on energy. More coming…"
3198,Sal from www.khanacademy.org came by to meet with us. A short video on the gates notes website - http://bit.ly/98sHQw


In [53]:
import re
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

# Clean the tweets
tweet['tweet'] = tweet['tweet'].apply(cleanTxt)

j =1
sentence = []
for i in tweet['tweet']:
    i = i.replace('.','.\n')
    separated_string = i.splitlines()
    for k in separated_string:
        if not k.strip():
                continue
        else:
            sentence.append(k)
        j = j+1

tweetdata = pd.DataFrame(sentence,columns = ['tweet'])

In [54]:
pd.options.display.max_colwidth = 200
tweetdata['tweet'].to_frame()

,tweet
0,"Here in Washington, we always vote by mail."
1,It's a safe and secure way to shape the future of our country.
2,But rega…
3,The UAE is an essential partner in stopping polio in Pakistan.
4,Thank you for your…
...,...
5510,khanacademy.
5511,org came by to meet with us.
5512,A short video on the gates notes website -
5513,View of a small section of the BYD Shenzhen plant.


In [55]:
from textblob import TextBlob

def populateSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def populatePolarity(text):
    return TextBlob(text).sentiment.polarity

tweetdata['Subjectivity'] = tweetdata.tweet.apply(populateSubjectivity)
tweetdata['polarity'] = tweetdata.tweet.apply(populatePolarity)
tweetdata

,tweet,Subjectivity,polarity
0,"Here in Washington, we always vote by mail.",0.000000,0.00
1,It's a safe and secure way to shape the future of our country.,0.408333,0.30
2,But rega…,0.000000,0.00
3,The UAE is an essential partner in stopping polio in Pakistan.,0.300000,0.00
4,Thank you for your…,0.000000,0.00
...,...,...,...
5510,khanacademy.,0.000000,0.00
5511,org came by to meet with us.,0.000000,0.00
5512,A short video on the gates notes website -,0.300000,0.00
5513,View of a small section of the BYD Shenzhen plant.,0.400000,-0.25


In [56]:
def result(Score):
    if Score<0:
        return 0
    else: 
        return 1
tweetdata['label'] = tweetdata.polarity.apply(result)
tweetdata

,tweet,Subjectivity,polarity,label
0,"Here in Washington, we always vote by mail.",0.000000,0.00,1
1,It's a safe and secure way to shape the future of our country.,0.408333,0.30,1
2,But rega…,0.000000,0.00,1
3,The UAE is an essential partner in stopping polio in Pakistan.,0.300000,0.00,1
4,Thank you for your…,0.000000,0.00,1
...,...,...,...,...
5510,khanacademy.,0.000000,0.00,1
5511,org came by to meet with us.,0.000000,0.00,1
5512,A short video on the gates notes website -,0.300000,0.00,1
5513,View of a small section of the BYD Shenzhen plant.,0.400000,-0.25,0


In [27]:
from __future__ import print_function, division, unicode_literals

import numbers
import numpy as np

from torchmoji.create_vocab import extend_vocab, VocabBuilder
from torchmoji.word_generator import WordGenerator
from torchmoji.global_variables import SPECIAL_TOKENS

# import torch

from sklearn.model_selection import train_test_split

from copy import deepcopy

class SentenceTokenizer_validation():
    """ Create numpy array of tokens corresponding to input sentences.
        The vocabulary can include Unicode tokens.
    """
    def __init__(self, vocabulary, fixed_length, custom_wordgen=None,
                 ignore_sentences_with_only_custom=False, masking_value=0,
                 unknown_value=1):
        """ Needs a dictionary as input for the vocabulary.
        """

        if len(vocabulary) > np.iinfo('uint16').max:
            raise ValueError('Dictionary is too big ({} tokens) for the numpy '
                             'datatypes used (max limit={}). Reduce vocabulary'
                             ' or adjust code accordingly!'
                             .format(len(vocabulary), np.iinfo('uint16').max))

        # Shouldn't be able to modify the given vocabulary
        self.vocabulary = deepcopy(vocabulary)
        self.fixed_length = fixed_length
        self.ignore_sentences_with_only_custom = ignore_sentences_with_only_custom
        self.masking_value = masking_value
        self.unknown_value = unknown_value

        # Initialized with an empty stream of sentences that must then be fed
        # to the generator at a later point for reusability.
        # A custom word generator can be used for domain-specific filtering etc
        if custom_wordgen is not None:
            assert custom_wordgen.stream is None
            self.wordgen = custom_wordgen
            self.uses_custom_wordgen = True
        else:
            self.wordgen = WordGenerator(None, allow_unicode_text=True,
                                         ignore_emojis=False,
                                         remove_variation_selectors=True,
                                         break_replacement=True)
            self.uses_custom_wordgen = False

    def tokenize_sentences(self, sentences, reset_stats=True, max_sentences=None):

        if max_sentences is None and not hasattr(sentences, '__len__'):
            raise ValueError('Either you must provide an array with a length'
                             'attribute (e.g. a list) or specify the maximum '
                             'length yourself using `max_sentences`!')
        n_sentences = (max_sentences if max_sentences is not None
                       else len(sentences))

        if self.masking_value == 0:
            tokens = np.zeros((n_sentences, self.fixed_length), dtype='uint16')
        else:
            tokens = (np.ones((n_sentences, self.fixed_length), dtype='uint16')
                      * self.masking_value)

        if reset_stats:
            self.wordgen.reset_stats()

        # With a custom word generator info can be extracted from each
        # sentence (e.g. labels)
        infos = []

        # Returns words as strings and then map them to vocabulary
        self.wordgen.stream = sentences
        next_insert = 0
        n_ignored_unknowns = 0
        for s_words, s_info in self.wordgen:
            s_tokens = self.find_tokens(s_words)

            if (self.ignore_sentences_with_only_custom and
                np.all([True if t < len(SPECIAL_TOKENS)
                        else False for t in s_tokens])):
                n_ignored_unknowns += 1
                continue
            if len(s_tokens) > self.fixed_length:
                s_tokens = s_tokens[:self.fixed_length]
            tokens[next_insert,:len(s_tokens)] = s_tokens
            infos.append(s_info)
            next_insert += 1

        # For standard word generators all sentences should be tokenized
        # this is not necessarily the case for custom wordgenerators as they
        # may filter the sentences etc.
        if not self.uses_custom_wordgen and not self.ignore_sentences_with_only_custom:
            assert len(sentences) == next_insert
        else:
            # adjust based on actual tokens received
            tokens = tokens[:next_insert]
            infos = infos[:next_insert]

        return tokens, infos, self.wordgen.stats

    def find_tokens(self, words):
        assert len(words) > 0
        tokens = []
        for w in words:
            try:
                tokens.append(self.vocabulary[w])
            except KeyError:
                tokens.append(self.unknown_value)
        return tokens
    
    def split_train_val_test(self, sentences, info_dicts,
                             split_parameter=[0], extend_with=0):
        if isinstance(split_parameter, list) and \
                all(isinstance(x, list) for x in split_parameter) and \
                len(split_parameter) == 3:
            # Helper function to verify provided indices are numbers in range
            def verify_indices(inds):
                return list(filter(lambda i: isinstance(i, numbers.Number)
                            and i < len(sentences), inds))
            ind_val = verify_indices(split_parameter[0])
        else:
            ind_val = list(range(len(sentences)))
            
        val = np.array([sentences[x] for x in ind_val])
        info_val = np.array([info_dicts[x] for x in ind_val])

        added = 0
        # Extend vocabulary with training set tokens
        if extend_with > 0:
            wg = WordGenerator(val)
            vb = VocabBuilder(wg)
            vb.count_all_words()
            added = extend_vocab(self.vocabulary, vb, max_tokens=extend_with)

        # Wrap results
        result = [self.tokenize_sentences(s)[0] for s in [val]]
        result_infos = [info_val]
        # if type(result_infos[0][0]) in [np.double, np.float, np.int64, np.int32, np.uint8]:
        #     result_infos = [torch.from_numpy(label).long() for label in result_infos]

        return result, result_infos, added

In [28]:
def load_benchmarktest(vocab, extend_with=0):
    batch_size, maxlen = calculate_batchsize_maxlen(tweetdata['tweet'])
    st = SentenceTokenizer_validation(vocab, maxlen)
    texts, labels, added = st.split_train_val_test(tweetdata['tweet'],
                                                   tweetdata['label'],
                                                   extend_with=extend_with)
    return {'texts': texts,
            'labels': labels,
            'added': added,
            'batch_size': batch_size,
            'maxlen': maxlen}

In [29]:
from torchmoji.tokenizer import tokenize
import math
def calculate_batchsize_maxlen(texts):
    def roundup(x):
        return int(math.ceil(x / 10.0)) * 10
    lengths = [len(tokenize(t)) for t in texts]
    maxlen = roundup(np.percentile(lengths, 80.0))
    batch_size = 250 if maxlen <= 100 else 50
    return batch_size, maxlen

In [30]:
import torch
def evaluate_using_acc(model, test_gen):
    # Validate on test_data
    model.eval()
    accs = []
    output = []
    for i, data in enumerate(test_gen):
        x, y = data
        outs = model(x)
        if model.nb_classes > 2:
            pred = torch.max(outs, 1)[1]
            acc = accuracy_score(y.squeeze().numpy(), pred.squeeze().numpy())
        else:
            pred = torch.argmax(outs, 1)
            acc = (pred == y).double().sum() / len(pred)
            output.append(pred)
        accs.append(acc)
    return np.mean(accs),output

In [31]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore")
import json
from torchmoji.model_def import torchmoji_transfer
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH, ROOT_PATH
from torchmoji.finetuning import (finetune,get_data_loader)

with open(VOCAB_PATH, 'r') as f:
        vocab = json.load(f)
nb_classes = 2
# Load dataset.
test_gen = load_benchmarktest(vocab)
#test_gen
model = torchmoji_transfer(nb_classes, PRETRAINED_PATH)
test_gen = get_data_loader(test_gen['texts'][0], test_gen['labels'][0], 100,
                              extended_batch_sampler=False)
Acc, pred = evaluate_using_acc(model,test_gen)

print(Acc)

Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0.bias_ih_l0_reverse
Loading weights for lstm_0.bias_hh_l0_reverse
Loading weights for lstm_1.weight_ih_l0
Loading weights for lstm_1.weight_hh_l0
Loading weights for lstm_1.bias_ih_l0
Loading weights for lstm_1.bias_hh_l0
Loading weights for lstm_1.weight_ih_l0_reverse
Loading weights for lstm_1.weight_hh_l0_reverse
Loading weights for lstm_1.bias_ih_l0_reverse
Loading weights for lstm_1.bias_hh_l0_reverse
Loading weights for attention_layer.attention_vector
Ignoring weights for output_layer.0.weight
Ignoring weights for output_layer.0.bias
0.9220238095238095
